In [1]:
import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell
def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path
    
    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path
        
class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path
    
    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)
        
        print ("importing Jupyter notebook from %s" % path)
                                       
        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)
        
        
        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod
        
        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__
        
        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod
class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}
    
    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return
        
        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)
        
        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

sys.meta_path.append(NotebookFinder())

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np

In [3]:
import dataset

filename = './train.mat'
data,_ = dataset.getdata(filename)
data = pd.DataFrame(data)

importing Jupyter notebook from dataset.ipynb


In [4]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        
        # 定义编码器
        self.encoder = nn.Sequential(
            nn.Linear(66, 40),
            nn.Sigmoid(),
            nn.Linear(40, 25),
            nn.Sigmoid(),
            nn.Linear(25, 15),
            nn.Sigmoid())
        
        # 定义解码器
        self.decoder = nn.Sequential(
            nn.Linear(15, 25),
            nn.Sigmoid(),
            nn.Linear(25, 40),
            nn.Sigmoid(),
            nn.Linear(40, 66),
            nn.Sigmoid())

    def forward(self, x):
        # 将输入数据经过编码器和解码器处理，得到重构后的输出
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [5]:
# 创建自编码器模型实例
model = Autoencoder()
# 定义损失函数、优化器
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

X_train= data.values
# 自编码器训练
for epoch in range(50):
    running_loss = 0.0
    for i, data in enumerate(X_train):
        inputs = torch.FloatTensor(data)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, inputs)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print('Epoch %d loss: %.3f' % (epoch + 1, running_loss / i))

Epoch 1 loss: 4.408
Epoch 2 loss: 4.405
Epoch 3 loss: 4.403
Epoch 4 loss: 4.403
Epoch 5 loss: 4.403
Epoch 6 loss: 4.401
Epoch 7 loss: 4.400
Epoch 8 loss: 4.400
Epoch 9 loss: 4.400
Epoch 10 loss: 4.400
Epoch 11 loss: 4.400
Epoch 12 loss: 4.399
Epoch 13 loss: 4.399
Epoch 14 loss: 4.399
Epoch 15 loss: 4.399
Epoch 16 loss: 4.399
Epoch 17 loss: 4.399
Epoch 18 loss: 4.399
Epoch 19 loss: 4.399
Epoch 20 loss: 4.399
Epoch 21 loss: 4.399
Epoch 22 loss: 4.399
Epoch 23 loss: 4.399
Epoch 24 loss: 4.399
Epoch 25 loss: 4.399
Epoch 26 loss: 4.399
Epoch 27 loss: 4.399
Epoch 28 loss: 4.399
Epoch 29 loss: 4.399
Epoch 30 loss: 4.399
Epoch 31 loss: 4.399
Epoch 32 loss: 4.399
Epoch 33 loss: 4.399
Epoch 34 loss: 4.399
Epoch 35 loss: 4.399
Epoch 36 loss: 4.399
Epoch 37 loss: 4.399
Epoch 38 loss: 4.399
Epoch 39 loss: 4.399
Epoch 40 loss: 4.399
Epoch 41 loss: 4.399
Epoch 42 loss: 4.399
Epoch 43 loss: 4.399
Epoch 44 loss: 4.399
Epoch 45 loss: 4.399
Epoch 46 loss: 4.399
Epoch 47 loss: 4.399
Epoch 48 loss: 4.399
E

In [6]:
filename = './test.mat'
new_data, orgdata = dataset.getdata(filename)
k = 2
r = orgdata.shape[0]
c = orgdata.shape[1]

T = 600

err = np.zeros((T)) 
for i in range(T):
    org = orgdata[:,:,i]
    new = torch.tensor(new_data[i, :])
    new = new.float()
    decoded_data = model.forward(new)
    combined = decoded_data.detach().numpy()
    
    U = combined[:r*k].reshape(r,k)
    S = combined[r*k:r*k+k*k].reshape(k,k)
    V = combined[r*k+k*k:].reshape(k,c)
    
    for j in range(k):
        U[:,j]=U[:,j]*S[j,j]
    recover = np.dot(U,V)
    
    err[i] = ((recover - org) ** 2).mean()

In [7]:
z_scores = np.zeros((T)) 
mean = np.mean(err)
std = np.std(err)

# 计算 z-score
for  i in range(T):
    z_scores[i] = (err[i] - mean) / std